## Note - This example code uses a SQL Server instance running on a docker Linux container.  If you have created a container using the BuildContainer notebook, use these scripts as is.  If you are running on a native instance, modify the scripts according the comments.  

## Step 1 - Connect to your SQL Server instance using Azure Data Studio.

In [10]:
/* If you created a SQL container in the previous step, you can use this script as is.
   If you are using a different instance, change the FILENAME paths to be valid.
   Change the database and object names if you must.  */
USE master
GO
DROP DATABASE IF EXISTS whitesox
GO
CREATE DATABASE whitesox
ON PRIMARY
(NAME = N'whitesox_primary', FILENAME = '/var/opt/mssql/data/whitesox.mdf', SIZE = 2Gb , MAXSIZE = UNLIMITED, FILEGROWTH = 65536KB)
LOG ON 
(NAME = N'whitesox_Log', FILENAME = '/var/opt/mssql/data/whitesox_log.ldf', SIZE = 5Gb , MAXSIZE = UNLIMITED , FILEGROWTH = 65536KB)
GO
ALTER DATABASE whitesox SET RECOVERY SIMPLE
GO
USE whitesox
GO
DROP TABLE IF EXISTS letsgowhitesox
GO
CREATE TABLE letsgowhitesox (playerid int primary key clustered, playername char(7000) not null)
GO
SET NOCOUNT ON
GO
BEGIN TRAN
DECLARE @x int
SET @x = 0
WHILE (@x < 100000)
BEGIN
	INSERT INTO letsgowhitesox VALUES (@x, 'Gerrit Cole on the South Side!')
	SET @x = @x + 1
END
COMMIT TRAN
GO
SET NOCOUNT OFF
GO
USE master
GO

Query was canceled by user

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

...

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:29.435

## Step 2 - Delete all rows for the table in an open transaction.

In [2]:
USE master
GO
ALTER DATABASE whitesox SET ACCELERATED_DATABASE_RECOVERY = OFF
GO
USE whitesox
GO
BEGIN TRAN
DELETE from letsgowhitesox
GO

## Step 3 - Check transaction log usage.
### Because there is an open transaction, the log file will contain space that cannot be reused.

In [3]:
USE whitesox;

SELECT * FROM sys.dm_db_log_space_usage
GO

## Step 4 - Does a CHECKPOINT free log space
### Compare used_log_space_in_percent to the previous query.

In [4]:
USE whitesox;

CHECKPOINT
GO
SELECT * FROM sys.dm_db_log_space_usage
GO

## Step 5 - How long does it take to rollback the deletes?
### Rollback the transaction and check the duration.

In [5]:
USE whitesox;

ROLLBACK TRAN
GO

## Step 6 - Enable Accelerated Database Recovery

In [6]:
USE master
GO
ALTER DATABASE whitesox SET ACCELERATED_DATABASE_RECOVERY = ON
GO

## Step 7 - Delete all of the rows again within a transaction

In [7]:
USE whitesox
GO
BEGIN TRAN
DELETE from letsgowhitesox
GO

## Step 8 - Check log space used before and after a CHECKPOINT
### Notice log space usage is low even before the CHECKPOINT

In [8]:
USE whitesox;
SELECT * FROM sys.dm_db_log_space_usage
GO
CHECKPOINT
GO
SELECT * FROM sys.dm_db_log_space_usage
GO


## Step 9 - How fast is a ROLLBACK?
### Is ROLLBACK any faster with ADR enabled?

In [9]:
USE whitesox;

ROLLBACK TRAN
GO